# Gender classification

In [40]:
import sys,os
import numpy as np
import pandas as pd

## list some file directories

In [41]:

# Specify the directory
directory_clips = 'data/cv-corpus-16.1-delta-2023-12-06/en/clips'
directory_labels = 'data/cv-corpus-16.1-delta-2023-12-06/en/'

reported = 'data/cv-corpus-16.1-delta-2023-12-06/en/validated.tsv'

# List all files and directories in the specified directory
files = os.listdir(directory_clips)
labels = os.listdir(directory_labels)

# Print filenames
print(files[0:10])
print(labels[0:10])



['common_voice_en_39124688.mp3', 'common_voice_en_38492555.mp3', 'common_voice_en_39014458.mp3', 'common_voice_en_38627321.mp3', 'common_voice_en_39015882.mp3', 'common_voice_en_38559189.mp3', 'common_voice_en_39094136.mp3', 'common_voice_en_39583678.mp3', 'common_voice_en_38968922.mp3', 'common_voice_en_38743424.mp3']
['reported.tsv', 'clips', 'invalidated.tsv', 'clip_durations.tsv', 'validated.tsv', 'other.tsv']


### Inspect Labels

In [67]:
# load the data labels
df = pd.read_csv(reported, sep='\t', header=0,)

# drop unecesary columns
df = df. drop(columns=['client_id','up_votes', 'down_votes', 'accents', 'locale', 'segment', 'variant'])

# drop rows with missing values
df = df.dropna(subset=['gender'])

print(df.shape)

df.head()

(3076, 4)


,path,sentence,age,gender
5,common_voice_en_38570599.mp3,Flatow was given a co-star credit.,twenties,male
8,common_voice_en_39572424.mp3,A timetable for the move was not specified.,fifties,male
13,common_voice_en_39548774.mp3,Where can i find the supreme leader?,teens,female
15,common_voice_en_38594742.mp3,"Down jumped the driver, and out got Mr. Pickwick.",twenties,male
22,common_voice_en_38514380.mp3,He was the original sponsor of legislation cre...,sixties,male


## Data Import

## Data preperation

## Feature extraction